In [1]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,GlobalAveragePooling2D,Flatten
from sklearn.datasets import load_files
from keras.utils import np_utils
import numpy as np
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
bottleneck_train=np.load('bottleneck_train_resnet50_cancdetect')
bottleneck_valid=np.load('bottleneck_valid_resnet50_cancdetect')
bottleneck_test=np.load('bottleneck_test_resnet50_cancdetect')

In [3]:
bottleneck_train.shape

(2000, 1, 1, 2048)

In [4]:
trainfiles=load_files('data/train/')
train_filenames=trainfiles['filenames']
train_targets=np_utils.to_categorical(trainfiles['target'])

validfiles=load_files('data/valid/')
valid_filenames=validfiles['filenames']
valid_targets=np_utils.to_categorical(validfiles['target'])

testfiles=load_files('data/test/')
test_filenames=testfiles['filenames']
test_targets=np_utils.to_categorical(testfiles['target'])

In [5]:
model=Sequential()
model.add(GlobalAveragePooling2D(input_shape=bottleneck_train.shape[1:]))
model.add(Dense(units=len(trainfiles['target_names']),activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 6147      
Total params: 6,147.0
Trainable params: 6,147.0
Non-trainable params: 0.0
_________________________________________________________________


In [6]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='rmsprop')

In [7]:

checkpointer=ModelCheckpoint(filepath='cancdetect_transfer_resnet50.weights.best.hdf5',verbose=1,save_best_only=True)

model.fit(bottleneck_train,train_targets,batch_size=10,epochs=100,shuffle=True,verbose=1,callbacks=[checkpointer],
             validation_data=[bottleneck_valid,valid_targets])

Train on 2000 samples, validate on 146 samples
Epoch 1/100
2000/2000 [==============================] - 5s - loss: 0.8231 - acc: 0.6525 - val_loss: 0.8127 - val_acc: 0.7055
Epoch 2/100
2000/2000 [==============================] - 0s - loss: 0.6687 - acc: 0.7180 - val_loss: 0.7030 - val_acc: 0.7260
Epoch 3/100
2000/2000 [==============================] - 0s - loss: 0.5862 - acc: 0.7540 - val_loss: 0.7661 - val_acc: 0.7123
Epoch 4/100
2000/2000 [==============================] - 0s - loss: 0.5321 - acc: 0.7810 - val_loss: 0.7463 - val_acc: 0.7260
Epoch 5/100
2000/2000 [==============================] - 0s - loss: 0.5076 - acc: 0.7875 - val_loss: 0.7321 - val_acc: 0.7192
Epoch 6/100
2000/2000 [==============================] - 0s - loss: 0.4755 - acc: 0.8115 - val_loss: 1.0006 - val_acc: 0.6644
Epoch 7/100
2000/2000 [==============================] - 0s - loss: 0.4659 - acc: 0.8045 - val_loss: 0.8136 - val_acc: 0.6986
Epoch 8/100
2000/2000 [==============================] - 0s - loss: 0.4

In [8]:
model.load_weights('cancdetect_transfer_resnet50.weights.best.hdf5')
predictions=model.predict(bottleneck_test)

In [9]:
# ids=[filename[filename.index('ISIC'):] for filename in test_filenames]
ids=[filename for filename in test_filenames]
ids=[x.replace('\\','/') for x in ids]
sub_csv=[(x,y,z) for x,y,z in zip(ids,predictions[:,0],predictions[:,2])]

In [10]:
sub_csv=[('Id','task_1','task_2')]+sub_csv

In [11]:
import csv
with open('results_resnet50.csv', 'w',newline='')  as myfile:
    writer = csv.writer(myfile)
#     writer.writerow('file,species')
    writer.writerows(sub_csv)